In this notebook we give a proof of concept of unitary compiling using TFQ. 

In [87]:
%load_ext autoreload
%autoreload 2

import sympy 
import numpy as np 
import pandas as pd 
import tensorflow as tf
from utilities.circuit_database import CirqTranslater
from utilities.templates import *
from utilities.variational import Minimizer
from utilities.misc import get_qubits_involved, reindex_symbol, shift_symbols_down
import matplotlib.pyplot as plt 
import tensorflow_quantum as tfq
import cirq
from utilities.compiling import *
from utilities.misc import *
from utilities.simplifier import Simplifier
from utilities.discrimination import *
from utilities.idinserter import IdInserter
from utilities.evaluator import Evaluator
from utilities.gate_killer import GateKiller
from ast import literal_eval


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [88]:
np.random.seed(0)
translator = CirqTranslater(3, untouchable_blocks = [1])
translator.env_qubits = [2]
translator.encoder_id = 0
translator.channel_id = 1
translator.decoder_id = 2
simplifier = Simplifier(translator)
etas = [0.01, 1.]
minimizer = Minimizer(translator, mode="discrimination", params=etas)
killer = GateKiller(translator, mode="discrimination", params = etas)
inserter = IdInserter(translator.n_qubits, untouchable_blocks=translator.channel_id)
args_evaluator = {"n_qubits":translator.n_qubits, "problem":"acd","params":etas}
evaluator = Evaluator(args=args_evaluator, lower_bound_cost=minimizer.lower_bound_cost, nrun=0)

In [112]:
cdb = []
for ind, qubits in list(translator.indexed_cnots.items()):
    cdb.append(gate_template(int(ind), block_id=0))
    con, tar = qubits
    cdb.append(gate_template(int(con + translator.number_of_cnots), param_value = np.random.random()))
    cdb.append(gate_template(int(tar + translator.number_of_cnots + translator.n_qubits), param_value = np.random.random()))
circuit_db = pd.DataFrame(cdb)


In [113]:
circuit, circuit_db = translator.give_circuit(circuit_db)

In [114]:
circuit

(0, 0): ───@───Rz(th_0)───@───Rz(th_2)───X───Rx(th_5)──────────────────X───Rx(th_9)───────────────────
           │              │              │                             │
(0, 1): ───X───Rx(th_1)───┼──────────────@───Rz(th_4)───@───Rz(th_6)───┼──────────────X───Rx(th_11)───
                          │                             │              │              │
(0, 2): ──────────────────X───Rx(th_3)──────────────────X───Rx(th_7)───@───Rz(th_8)───@───Rz(th_10)───

In [115]:
simplified_db, ns = simplifier.reduce_circuit(circuit_db)
ss, simplified_db = translator.give_circuit(simplified_db)

<bound method Simplifier.rule_6 of <utilities.simplifier.Simplifier object at 0x7f593cc31a90>>
<bound method Simplifier.rule_6 of <utilities.simplifier.Simplifier object at 0x7f593cc31a90>>
<bound method Simplifier.rule_6 of <utilities.simplifier.Simplifier object at 0x7f593cc31a90>>
<bound method Simplifier.rule_6 of <utilities.simplifier.Simplifier object at 0x7f593cc31a90>>
<bound method Simplifier.rule_6 of <utilities.simplifier.Simplifier object at 0x7f593cc31a90>>


In [116]:
gates_on_qubit, on_qubit_order = simplifier.get_positional_dbs(circuit, circuit_db)
simplified_db = circuit_db.copy()

In [117]:
mdif = max_diff(translator , circuit_db, simplified_db)
mdif

0.0

In [59]:
simplification = False
for q, qubit_gates_path in gates_on_qubit.items():
    if simplification is True:
        break
    for order_gate_on_qubit, ind_gate in enumerate(qubit_gates_path[:-1]):
        if simplification is True:
            break
        ind_gate_p1 = qubit_gates_path[order_gate_on_qubit+1]
        ## if i have a rotation and then a CNOT
        if (check_rot(ind_gate, simplifier.translator) is True) and (check_cnot(ind_gate_p1, simplifier.translator) is True):
            type_0 = type_get(ind_gate, simplifier.translator)
            control, target = simplifier.translator.indexed_cnots[str(ind_gate_p1)]

            this_qubit = q
            other_qubits = [control, target]
            other_qubits.remove(q)
            other_qubit = other_qubits[0]

            ### now it happens two interesting things: type0 == 0 AND q == control
            ### or type_0 == 1 AND q == target  then swap orders
            if ((type_0 == 0) and (q == control)) or ((type_0 == 1) and (q == target)):
                if len(on_qubit_order[q])<2:
                    simplification=False
                else:
                    simplification = True
                    qq, order_gate_on_qubitt, indexx_rot = q, order_gate_on_qubit, index_rot
                    index_rot = on_qubit_order[q][order_gate_on_qubit]
                    info_rot = simplified_db.loc[index_rot].copy()
                break


In [67]:
simplified_db = simplified_db.drop(labels=[index_rot],axis=0)#

In [76]:
simplified_db.loc[on_qubit_order[qq][order_gate_on_qubitt+1 ] + 0.1] = info_rot

In [78]:
simplified_db = simplified_db.sort_index().reset_index(drop=True)

In [86]:
simplified_db = order_symbol_labels(simplified_db)

In [52]:
simplified_db = simplified_db.drop(labels=[index_rot],axis=0)#

simplified_db.loc[on_qubit_order[q][order_gate_on_qubit+1 ] + 0.1] = info_rot
simplified_db = simplified_db.sort_index().reset_index(drop=True)

True

In [54]:
circuit

(0, 0): ───@───Rz(th_0)───@───Rz(th_2)───
           │              │
(0, 1): ───X───Rx(th_1)───┼──────────────
                          │
(0, 2): ──────────────────X───Rx(th_3)───

In [ ]:
def rule_6(simplifier, simplified_db, on_qubit_order, gates_on_qubit):
    simplification = False
    for q, qubit_gates_path in gates_on_qubit.items():
        if simplification is True:
            break
        for order_gate_on_qubit, ind_gate in enumerate(qubit_gates_path[:-1]):
            if simplification is True:
                break
            ind_gate_p1 = qubit_gates_path[order_gate_on_qubit+1]
            ## if i have a rotation and then a CNOT
            if (check_rot(ind_gate, simplifier.translator) is True) and (check_cnot(ind_gate_p1, simplifier.translator) is True):
                type_0 = type_get(ind_gate, simplifier.translator)
                control, target = simplifier.translator.indexed_cnots[str(ind_gate_p1)]

                this_qubit = q
                other_qubits = [control, target]
                other_qubits.remove(q)
                other_qubit = other_qubits[0]

                ### now it happens two interesting things: type0 == 0 AND q == control
                ### or type_0 == 1 AND q == target  then swap orders

                if ((type_0 == 0) and (q == control)) or ((type_0 == 1) and (q == target)):
                    if len(on_qubit_order[q]) <2:
                        simplification=False
                    else:

                        simplification = True

                        ###now we swap the order in which we apply the rotation and the CNOT.
                        index_rot = on_qubit_order[q][order_gate_on_qubit]
                        info_rot = simplified_db.loc[index_rot].copy()
                        simplified_db = simplified_db.drop(labels=[index_rot],axis=0)#

                        simplified_db.loc[on_qubit_order[q][order_gate_on_qubit+1 ] + 0.1] = info_rot
                        simplified_db = simplified_db.sort_index().reset_index(drop=True)

                        break
    return simplification, simplified_db

In [45]:
simplified_db, ns = simplifier.reduce_circuit(circuit_db)
ss, simplified_db = translator.give_circuit(simplified_db)

<bound method Simplifier.rule_6 of <utilities.simplifier.Simplifier object at 0x7f5968265400>>
warning, repeated symbol while constructing the circuit, see circuut_
  symbol_name th_1
 symbols ['th_1', 'th_0']
circuit_db {0: {'ind': 0, 'symbol': None, 'param_value': None, 'trainable': True, 'block_id': 0}, 1: {'ind': 10, 'symbol': 'th_1', 'param_value': None, 'trainable': True, 'block_id': 0}, 2: {'ind': 1, 'symbol': None, 'param_value': None, 'trainable': True, 'block_id': 0}, 3: {'ind': 6, 'symbol': 'th_0', 'param_value': None, 'trainable': True, 'block_id': 0}, 4: {'ind': 6, 'symbol': 'th_1', 'param_value': None, 'trainable': True, 'block_id': 0}} 





In [5]:
block_0 = concatenate_dbs([z_layer_db(translator, block_id=0), z_layer_db(translator, block_id=0), z_layer_db(translator, block_id=0)])
blo=1
block_1 = concatenate_dbs([x_layer_db(translator, block_id=blo), x_layer_db(translator, block_id=blo), x_layer_db(translator, block_id=blo)])
circuit_db = concatenate_dbs([block_0, block_1])

In [6]:
circuit, circuit_db = translator.give_circuit(circuit_db)

In [7]:
simplified_db, ns = simplifier.reduce_circuit(circuit_db)
ss, simplified_db = translator.give_circuit(simplified_db)

<bound method Simplifier.rule_4 of <utilities.simplifier.Simplifier object at 0x7f81bc0215c0>>
<bound method Simplifier.rule_4 of <utilities.simplifier.Simplifier object at 0x7f81bc0215c0>>
<bound method Simplifier.rule_4 of <utilities.simplifier.Simplifier object at 0x7f81bc0215c0>>
<bound method Simplifier.rule_4 of <utilities.simplifier.Simplifier object at 0x7f81bc0215c0>>
<bound method Simplifier.rule_4 of <utilities.simplifier.Simplifier object at 0x7f81bc0215c0>>
<bound method Simplifier.rule_4 of <utilities.simplifier.Simplifier object at 0x7f81bc0215c0>>
<bound method Simplifier.rule_4 of <utilities.simplifier.Simplifier object at 0x7f81bc0215c0>>
<bound method Simplifier.rule_4 of <utilities.simplifier.Simplifier object at 0x7f81bc0215c0>>
<bound method Simplifier.rule_4 of <utilities.simplifier.Simplifier object at 0x7f81bc0215c0>>
<bound method Simplifier.rule_4 of <utilities.simplifier.Simplifier object at 0x7f81bc0215c0>>
<bound method Simplifier.rule_4 of <utilities.simp

In [8]:
mdif = max_diff(translator , circuit_db, simplified_db)
mdif

6.280369834735101e-16

In [10]:
translator.give_circuit(simplified_db)[0]

(0, 0): ───Rz(th_0)───Rx(th_3)───

(0, 1): ───Rz(th_1)───Rx(th_4)───

(0, 2): ───Rz(th_2)───Rx(th_5)───